In [1]:
# import face_recognition_api
import face_recognition.api as face_recognition_api
import cv2
import os
import pickle
import numpy as np
import warnings
from PIL import Image, ImageDraw, ImageFont
import time

In [2]:
# Load Face Recogniser classifier
fname = 'classifier.pkl'
if os.path.isfile(fname):
    with open(fname, 'rb') as f:
        (le, clf) = pickle.load(f)
else:
    print('\x1b[0;37;43m' + "Classifier '{}' does not exist".format(fname) + '\x1b[0m')
    quit()


In [3]:
# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True


# video = 'D:\\data\\video\\綜藝大集合 20170709 台南 安南.mp4'
# video = 'D:\\data\\video\\綜藝大集合20180930 新竹新豐 - 片段1.mp4'
# video = 'D:\\data\\video\\綜藝大集合20180930 新竹新豐.mp4'
video = 'D:\\data\\video\\綜藝大集合20181230 新竹市.mp4'
 

FILE_OUTPUT = video+'.output.avi'
print(FILE_OUTPUT)

D:\data\video\綜藝大集合20181230 新竹市.mp4.output.avi


# 直接輸出到影像檔案中(存檔)

In [4]:
# Checks and deletes the output file
# You cant have a existing file or it will through an error
if os.path.isfile(FILE_OUTPUT):
    os.remove(FILE_OUTPUT)
    print('remove output file')

#####################
stime = time.localtime()
print(time.strftime("%Y-%m-%d %H:%M:%S",stime))
#####################
    
# Playing video from file:
cap = cv2.VideoCapture(video)
currentFrame, j = 0, 2
# Get current width of frame
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
# Get current height of frame
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # float
print('width, height',width, height)

fps = cap.get(cv2.CAP_PROP_FRAME_COUNT) # float
print(fps)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(FILE_OUTPUT,fourcc, 29.0, (int(width),int(height)))


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
# while(True):
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret == True:
            if currentFrame % j == 0:
#             if process_this_frame:    
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25, interpolation=cv2.INTER_CUBIC)
#                 small_frame = cv2.resize(frame, (0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_CUBIC)
#                 small_frame = cv2.resize(frame, (0, 0), fx=1.0, fy=1.0, interpolation=cv2.INTER_CUBIC)
                print('currentFrame : ', currentFrame)
#                 face_locations = face_recognition_api.face_locations(small_frame, model='cnn')
                # 一次只能處理一張圖，所以不能做多線程，因為串流一次吃進一張圖處理完後輸出
                # number_of_times_to_upsample 這只能調整框臉的數量(數字越大可框出越多小臉)
                face_locations = face_recognition_api.face_locations(small_frame, number_of_times_to_upsample=1)
                face_encodings = face_recognition_api.face_encodings(small_frame, face_locations)  # 這很慢
            
                face_names = []
                predictions = []
#                 predictions = face_locations
                if len(face_encodings) > 0:
                    closest_distances = clf.kneighbors(face_encodings, n_neighbors=1)
                    is_recognized = [closest_distances[0][i][0] <= 0.3999 for i in range(len(face_locations))]
                
                    predictions = [(le.inverse_transform([int(pred)])[0], loc) if rec else ("　　　", loc) for \
                            pred, loc, rec in zip(clf.predict(face_encodings), face_locations, is_recognized)]
                    
                    print('predictions ',predictions)
                    print('closest_distances ',closest_distances)
#             process_this_frame = not process_this_frame
    
            for name, (top, right, bottom, left) in predictions:
#             for (top, right, bottom, left) in predictions:
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                if name != "　　　":
#                     cv2.rectangle(frame, (left, bottom - 30), (right, bottom), (0, 0, 255), cv2.FILLED)
#                     pil_im = Image.fromarray(frame)
#                     draw = ImageDraw.Draw(pil_im)
#                     font = ImageFont.truetype('C:\\Windows\\Fonts\\mingliu.ttc', 30, encoding='utf-8')   #, encoding='utf-8'
#                     draw.text((left, bottom - 30), name, (255, 255, 255), font=font)
#                     frame = np.array(pil_im)
                    
                    
#                     cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    cv2.rectangle(frame, (left, bottom - 30), (right, bottom), (0, 0, 255), cv2.FILLED)
                    pil_im = Image.fromarray(frame)
                    draw = ImageDraw.Draw(pil_im)
                    font = ImageFont.truetype('C:\\Windows\\Fonts\\simsun.ttc',30,encoding='utf-8')
                    draw.text((left, bottom - 30), name, (255, 255, 255), font=font)
                    frame = np.array(pil_im)
                    
                    
                else:
                    pass
                

#                     predictions = []
            

            # Saves for video
            out.write(frame)

            # Display the resulting frame
            cv2.imshow('frame',frame)
        else:
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # To stop duplicate images
        currentFrame += 1

# When everything done, release the capture
print('process finished on Frame ...' , currentFrame)
cap.release()
out.release()
cv2.destroyAllWindows()

etime = time.localtime()
print(time.strftime("%Y-%m-%d %H:%M:%S",etime))

2019-01-25 21:34:16
width, height 1280.0 720.0
5395.0
currentFrame :  0
predictions  [('\u3000\u3000\u3000', (74, 201, 110, 165))]
closest_distances  (array([[0.6684226]]), array([[3]], dtype=int64))
currentFrame :  2
predictions  [('\u3000\u3000\u3000', (74, 197, 110, 161)), ('\u3000\u3000\u3000', (58, 241, 94, 205))]
closest_distances  (array([[0.66214586],
       [0.4040377 ]]), array([[8],
       [7]], dtype=int64))
currentFrame :  4
predictions  [('\u3000\u3000\u3000', (78, 197, 114, 161)), ('阿翔 A Xiang', (58, 241, 94, 205))]
closest_distances  (array([[0.66614607],
       [0.3878305 ]]), array([[8],
       [7]], dtype=int64))
currentFrame :  6
predictions  [('\u3000\u3000\u3000', (78, 197, 114, 161)), ('阿翔 A Xiang', (58, 245, 94, 209)), ('\u3000\u3000\u3000', (54, 221, 90, 185))]
closest_distances  (array([[0.66537592],
       [0.39156412],
       [0.69352074]]), array([[3],
       [7],
       [8]], dtype=int64))
currentFrame :  8
predictions  [('\u3000\u3000\u3000', (78, 197, 11

predictions  [('\u3000\u3000\u3000', (78, 209, 114, 173)), ('\u3000\u3000\u3000', (34, 165, 70, 129)), ('\u3000\u3000\u3000', (46, 133, 82, 97)), ('\u3000\u3000\u3000', (50, 108, 94, 64))]
closest_distances  (array([[0.68940568],
       [0.56690765],
       [0.56968291],
       [0.44552853]]), array([[0],
       [8],
       [1],
       [0]], dtype=int64))
currentFrame :  56
predictions  [('\u3000\u3000\u3000', (78, 209, 114, 173)), ('\u3000\u3000\u3000', (34, 165, 70, 129)), ('\u3000\u3000\u3000', (46, 133, 82, 97))]
closest_distances  (array([[0.68504807],
       [0.60219786],
       [0.71306373]]), array([[8],
       [8],
       [8]], dtype=int64))
currentFrame :  58
predictions  [('\u3000\u3000\u3000', (78, 209, 114, 173)), ('\u3000\u3000\u3000', (34, 165, 70, 129)), ('\u3000\u3000\u3000', (49, 129, 101, 78)), ('\u3000\u3000\u3000', (62, 245, 98, 209))]
closest_distances  (array([[0.73156583],
       [0.57448491],
       [0.44145861],
       [0.57912722]]), array([[0],
       [8],
 

predictions  [('\u3000\u3000\u3000', (74, 209, 110, 173)), ('\u3000\u3000\u3000', (54, 249, 90, 213))]
closest_distances  (array([[0.53133214],
       [0.6587625 ]]), array([[8],
       [7]], dtype=int64))
currentFrame :  110
predictions  [('\u3000\u3000\u3000', (74, 213, 110, 177)), ('\u3000\u3000\u3000', (54, 253, 90, 217))]
closest_distances  (array([[0.55447954],
       [0.68637303]]), array([[8],
       [7]], dtype=int64))
currentFrame :  112
predictions  [('\u3000\u3000\u3000', (74, 213, 110, 177)), ('\u3000\u3000\u3000', (54, 253, 90, 217))]
closest_distances  (array([[0.48776667],
       [0.69281227]]), array([[8],
       [7]], dtype=int64))
currentFrame :  114
predictions  [('\u3000\u3000\u3000', (74, 213, 110, 177)), ('\u3000\u3000\u3000', (50, 253, 86, 217))]
closest_distances  (array([[0.50771339],
       [0.69863028]]), array([[8],
       [7]], dtype=int64))
currentFrame :  116
predictions  [('\u3000\u3000\u3000', (74, 217, 110, 181)), ('\u3000\u3000\u3000', (50, 253, 86, 

predictions  [('\u3000\u3000\u3000', (54, 297, 90, 261)), ('\u3000\u3000\u3000', (42, 157, 78, 121)), ('\u3000\u3000\u3000', (74, 213, 110, 177))]
closest_distances  (array([[0.78184604],
       [0.54362692],
       [0.66116291]]), array([[0],
       [8],
       [0]], dtype=int64))
currentFrame :  164
predictions  [('\u3000\u3000\u3000', (54, 297, 90, 261)), ('\u3000\u3000\u3000', (42, 157, 78, 121)), ('\u3000\u3000\u3000', (74, 213, 110, 177))]
closest_distances  (array([[0.73964719],
       [0.54124018],
       [0.61423315]]), array([[1],
       [8],
       [8]], dtype=int64))
currentFrame :  166
predictions  [('\u3000\u3000\u3000', (42, 157, 78, 121)), ('\u3000\u3000\u3000', (54, 297, 90, 261)), ('\u3000\u3000\u3000', (74, 213, 110, 177))]
closest_distances  (array([[0.56360441],
       [0.76043361],
       [0.66754954]]), array([[8],
       [1],
       [1]], dtype=int64))
currentFrame :  168
predictions  [('\u3000\u3000\u3000', (54, 301, 90, 265)), ('\u3000\u3000\u3000', (42, 157, 

predictions  [('\u3000\u3000\u3000', (54, 309, 90, 273)), ('\u3000\u3000\u3000', (42, 161, 78, 125)), ('阿翔 A Xiang', (62, 233, 98, 197))]
closest_distances  (array([[0.76307485],
       [0.53346459],
       [0.20233023]]), array([[0],
       [8],
       [8]], dtype=int64))
currentFrame :  218
predictions  [('\u3000\u3000\u3000', (54, 309, 90, 273)), ('\u3000\u3000\u3000', (42, 157, 78, 121)), ('阿翔 A Xiang', (62, 229, 98, 193))]
closest_distances  (array([[0.78173134],
       [0.51099778],
       [0.17205698]]), array([[0],
       [8],
       [8]], dtype=int64))
currentFrame :  220
predictions  [('\u3000\u3000\u3000', (54, 309, 90, 273)), ('\u3000\u3000\u3000', (42, 161, 78, 125)), ('阿翔 A Xiang', (62, 229, 98, 193)), ('\u3000\u3000\u3000', (46, 41, 82, 5))]
closest_distances  (array([[0.77129559],
       [0.49464918],
       [0.20862853],
       [0.71326388]]), array([[0],
       [8],
       [8],
       [8]], dtype=int64))
currentFrame :  222
predictions  [('\u3000\u3000\u3000', (54, 30

...

currentFrame :  4314
predictions  [('\u3000\u3000\u3000', (41, 165, 84, 122))]
closest_distances  (array([[0.51059772]]), array([[1]], dtype=int64))
currentFrame :  4316
predictions  [('\u3000\u3000\u3000', (41, 165, 84, 122))]
closest_distances  (array([[0.40307137]]), array([[1]], dtype=int64))
currentFrame :  4318
predictions  [('\u3000\u3000\u3000', (41, 165, 84, 122))]
closest_distances  (array([[0.41884197]]), array([[0]], dtype=int64))
currentFrame :  4320
predictions  [('\u3000\u3000\u3000', (41, 165, 84, 122))]
closest_distances  (array([[0.46610773]]), array([[0]], dtype=int64))
currentFrame :  4322
predictions  [('\u3000\u3000\u3000', (41, 165, 84, 122))]
closest_distances  (array([[0.47515632]]), array([[1]], dtype=int64))
currentFrame :  4324
predictions  [('胡瓜 Hu gua', (46, 165, 89, 122))]
closest_distances  (array([[0.37905071]]), array([[1]], dtype=int64))
currentFrame :  4326
predictions  [('\u3000\u3000\u3000', (46, 165, 89, 122))]
closest_distances  (array([[0.415128

predictions  [('\u3000\u3000\u3000', (38, 233, 74, 197)), ('\u3000\u3000\u3000', (18, 185, 54, 149))]
closest_distances  (array([[0.52388464],
       [0.54023838]]), array([[0],
       [7]], dtype=int64))
currentFrame :  4402
predictions  [('\u3000\u3000\u3000', (38, 233, 74, 197))]
closest_distances  (array([[0.53007313]]), array([[0]], dtype=int64))
currentFrame :  4404
predictions  [('\u3000\u3000\u3000', (38, 233, 74, 197))]
closest_distances  (array([[0.51656391]]), array([[0]], dtype=int64))
currentFrame :  4406
predictions  [('\u3000\u3000\u3000', (38, 233, 74, 197))]
closest_distances  (array([[0.51860187]]), array([[0]], dtype=int64))
currentFrame :  4408
predictions  [('\u3000\u3000\u3000', (38, 233, 74, 197))]
closest_distances  (array([[0.53837647]]), array([[0]], dtype=int64))
currentFrame :  4410
predictions  [('\u3000\u3000\u3000', (38, 237, 74, 201))]
closest_distances  (array([[0.51141253]]), array([[0]], dtype=int64))
currentFrame :  4412
predictions  [('\u3000\u3000\

predictions  [('\u3000\u3000\u3000', (42, 225, 78, 189)), ('\u3000\u3000\u3000', (54, 97, 90, 61))]
closest_distances  (array([[0.47689481],
       [0.5435229 ]]), array([[8],
       [1]], dtype=int64))
currentFrame :  4520
predictions  [('\u3000\u3000\u3000', (42, 225, 78, 189)), ('\u3000\u3000\u3000', (54, 97, 90, 61))]
closest_distances  (array([[0.48801734],
       [0.54351582]]), array([[8],
       [0]], dtype=int64))
currentFrame :  4522
predictions  [('\u3000\u3000\u3000', (42, 225, 78, 189)), ('\u3000\u3000\u3000', (54, 97, 90, 61)), ('胡瓜 Hu gua', (38, 152, 90, 101))]
closest_distances  (array([[0.48512332],
       [0.54329405],
       [0.36527215]]), array([[8],
       [3],
       [1]], dtype=int64))
currentFrame :  4524
predictions  [('\u3000\u3000\u3000', (42, 217, 78, 181)), ('\u3000\u3000\u3000', (54, 93, 90, 57))]
closest_distances  (array([[0.5198555 ],
       [0.53186233]]), array([[8],
       [0]], dtype=int64))
currentFrame :  4526
predictions  [('\u3000\u3000\u3000',

currentFrame :  4620
predictions  [('\u3000\u3000\u3000', (42, 201, 78, 165))]
closest_distances  (array([[0.47049044]]), array([[8]], dtype=int64))
currentFrame :  4622
predictions  [('\u3000\u3000\u3000', (42, 201, 78, 165))]
closest_distances  (array([[0.47438827]]), array([[8]], dtype=int64))
currentFrame :  4624
predictions  [('\u3000\u3000\u3000', (42, 197, 78, 161))]
closest_distances  (array([[0.49700333]]), array([[8]], dtype=int64))
currentFrame :  4626
predictions  [('\u3000\u3000\u3000', (42, 197, 78, 161))]
closest_distances  (array([[0.48579491]]), array([[8]], dtype=int64))
currentFrame :  4628
predictions  [('\u3000\u3000\u3000', (42, 197, 78, 161))]
closest_distances  (array([[0.47497892]]), array([[8]], dtype=int64))
currentFrame :  4630
predictions  [('\u3000\u3000\u3000', (42, 197, 78, 161))]
closest_distances  (array([[0.47579047]]), array([[8]], dtype=int64))
currentFrame :  4632
predictions  [('\u3000\u3000\u3000', (42, 197, 78, 161))]
closest_distances  (array([

predictions  [('\u3000\u3000\u3000', (50, 85, 86, 49)), ('\u3000\u3000\u3000', (34, 205, 70, 169)), ('\u3000\u3000\u3000', (58, 125, 94, 89))]
closest_distances  (array([[0.53310302],
       [0.51681677],
       [0.58083728]]), array([[0],
       [8],
       [3]], dtype=int64))
currentFrame :  4804
predictions  [('\u3000\u3000\u3000', (50, 85, 86, 49)), ('\u3000\u3000\u3000', (38, 205, 74, 169)), ('\u3000\u3000\u3000', (62, 125, 98, 89))]
closest_distances  (array([[0.55379814],
       [0.54037801],
       [0.55933408]]), array([[0],
       [8],
       [3]], dtype=int64))
currentFrame :  4806
predictions  [('\u3000\u3000\u3000', (50, 85, 86, 49)), ('\u3000\u3000\u3000', (34, 205, 70, 169)), ('\u3000\u3000\u3000', (62, 121, 98, 85))]
closest_distances  (array([[0.56649474],
       [0.52045861],
       [0.59232163]]), array([[0],
       [8],
       [3]], dtype=int64))
currentFrame :  4808
predictions  [('\u3000\u3000\u3000', (34, 205, 70, 169)), ('\u3000\u3000\u3000', (50, 85, 86, 49)), 

In [5]:
# face_recognition_api.face_locations?